In [ ]:
import pygame
import random
import time
import sys

# Initialize Pygame
pygame.init()

# Set up display
WIDTH, HEIGHT = 800, 600
GRID_SIZE = 20
GRID_WIDTH = WIDTH // GRID_SIZE
GRID_HEIGHT = HEIGHT // GRID_SIZE
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Snake Game")

# Colors
BLACK = (0, 0, 0)
GREEN = (0, 255, 0)
DARK_GREEN = (0, 200, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
GRAY = (128, 128, 128)
WHITE = (255, 255, 255)

# Directions
UP = (0, -1)
DOWN = (0, 1)
LEFT = (-1, 0)
RIGHT = (1, 0)

# Font
font = pygame.font.Font(None, 36)
big_font = pygame.font.Font(None, 64)

# Game variables
base_speed = 10
snake = None
blue_berry = None
red_berry = None
bombs = []
level = 1
score = 0
game_over = False
paused = False

# Snake class
class Snake:
    def __init__(self):
        self.reset()

    def reset(self):
        self.positions = [(GRID_WIDTH // 2, GRID_HEIGHT // 2)]
        self.direction = random.choice([UP, DOWN, LEFT, RIGHT])
        self.length = 1

    def move(self):
        head = self.positions[0]
        x, y = self.direction
        new_head = ((head[0] + x) % GRID_WIDTH, (head[1] + y) % GRID_HEIGHT)

        if new_head in self.positions[2:] or new_head in bombs:
            return False

        self.positions.insert(0, new_head)
        if len(self.positions) > self.length:
            self.positions.pop()
        return True

    def render(self, surface):
        for i, pos in enumerate(self.positions):
            color = DARK_GREEN if i == 0 else GREEN
            pygame.draw.rect(surface, color, (pos[0] * GRID_SIZE, pos[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE), border_radius=5)

# Helper functions
def draw_grid(surface):
    for x in range(0, WIDTH, GRID_SIZE):
        pygame.draw.line(surface, (40, 40, 40), (x, 0), (x, HEIGHT))
    for y in range(0, HEIGHT, GRID_SIZE):
        pygame.draw.line(surface, (40, 40, 40), (0, y), (WIDTH, y))

def spawn_food():
    global blue_berry, red_berry
    while blue_berry is None:
        p = (random.randint(0, GRID_WIDTH - 1), random.randint(0, GRID_HEIGHT - 1))
        if p not in snake.positions and p not in bombs:
            blue_berry = p
    if red_berry is None and random.random() < 0.3:
        while True:
            p = (random.randint(0, GRID_WIDTH - 1), random.randint(0, GRID_HEIGHT - 1))
            if p not in snake.positions and p != blue_berry and p not in bombs:
                red_berry = p
                break

def spawn_bombs():
    global bombs
    bombs.clear()
    num = level * 2
    for _ in range(num):
        while True:
            p = (random.randint(0, GRID_WIDTH - 1), random.randint(0, GRID_HEIGHT - 1))
            if p not in snake.positions and p != blue_berry and p != red_berry:
                bombs.append(p)
                break

def next_level():
    global level, base_speed
    level += 1
    if level > 5:
        return True
    snake.reset()
    base_speed += 2
    spawn_bombs()
    return False

def reset_game():
    global snake, blue_berry, red_berry, bombs, level, score, base_speed, game_over, paused
    snake = Snake()
    blue_berry = None
    red_berry = None
    level = 1
    score = 0
    base_speed = 10
    game_over = False
    paused = False
    spawn_food()
    spawn_bombs()

# Main
clock = pygame.time.Clock()
reset_game()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP and snake.direction != DOWN:
                snake.direction = UP
            elif event.key == pygame.K_DOWN and snake.direction != UP:
                snake.direction = DOWN
            elif event.key == pygame.K_LEFT and snake.direction != RIGHT:
                snake.direction = LEFT
            elif event.key == pygame.K_RIGHT and snake.direction != LEFT:
                snake.direction = RIGHT
            elif event.key == pygame.K_r and game_over:
                reset_game()
            elif event.key == pygame.K_p and not game_over:
                paused = not paused

    if not paused and not game_over:
        if not snake.move():
            game_over = True

        head = snake.positions[0]
        if head == blue_berry:
            snake.length += 1
            score += 10
            blue_berry = None
            spawn_food()
            if score % 50 == 0:
                if next_level():
                    game_over = True
        elif head == red_berry:
            game_over = True

    # Draw
    screen.fill(BLACK)
    draw_grid(screen)
    snake.render(screen)

    if blue_berry:
        pygame.draw.rect(screen, BLUE, (blue_berry[0] * GRID_SIZE, blue_berry[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE), border_radius=5)
    if red_berry:
        pygame.draw.rect(screen, RED, (red_berry[0] * GRID_SIZE, red_berry[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE), border_radius=5)

    for b in bombs:
        pygame.draw.rect(screen, GRAY, (b[0] * GRID_SIZE, b[1] * GRID_SIZE, GRID_SIZE, GRID_SIZE), border_radius=5)

    # UI
    screen.blit(font.render(f"Score: {score}", True, WHITE), (10, 10))
    screen.blit(font.render(f"Level: {level}", True, WHITE), (10, 40))
    if paused:
        pause_text = big_font.render("PAUSED", True, WHITE)
        screen.blit(pause_text, (WIDTH // 2 - pause_text.get_width() // 2, HEIGHT // 2 - pause_text.get_height() // 2))

    if game_over:
        flash = RED if int(time.time() * 2) % 2 == 0 else WHITE
        over_text = font.render("Game Over! Press R to Restart", True, flash)
        final_text = font.render(f"Final Score: {score}", True, WHITE)
        screen.blit(over_text, (WIDTH // 2 - over_text.get_width() // 2, HEIGHT // 2 - 20))
        screen.blit(final_text, (WIDTH // 2 - final_text.get_width() // 2, HEIGHT // 2 + 20))

    pygame.display.flip()
    clock.tick(base_speed)

pygame.quit()

pygame 2.6.1 (SDL 2.28.4, Python 3.12.7)
Hello from the pygame community. https://www.pygame.org/contribute.html
